In [5]:
# 查看当前kernel下已安装的包  list packages
!pip list --format=columns

Package                   Version
------------------------- --------------
anyio                     4.7.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
comm                      0.2.2
debugpy                   1.8.9
decorator                 5.1.1
defusedxml                0.7.1
exceptiongroup            1.2.2
executing                 2.1.0
fastjsonschema            2.21.1
filelock                  3.16.1
fqdn                      1.5.1
fsspec                    2024.10.0
gensim                    4.3.3
h11                       0.14.0
httpcore                  1.0.7
httpx                     0.28.1
huggingface

In [6]:
# 安装运行代码所需要的第三方包
!pip install jieba --user
!pip install gensim --user
!pip install dataclasses --user

In [7]:
# 下载腾讯Lab词向量(v0.2.0, 100维-Small)
!wget https://ai.tencent.com/ailab/nlp/zh/data/tencent-ailab-embedding-zh-d100-v0.2.0-s.tar.gz
!tar -zxvf tencent-ailab-embedding-zh-d100-v0.2.0-s.tar.gz

--2024-12-09 17:10:24--  https://ai.tencent.com/ailab/nlp/zh/data/tencent-ailab-embedding-zh-d100-v0.2.0-s.tar.gz
正在解析主机 ai.tencent.com (ai.tencent.com)... 163.177.4.52
正在连接 ai.tencent.com (ai.tencent.com)|163.177.4.52|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：6840 (6.7K) [text/html]
正在保存至: “tencent-ailab-embedding-zh-d100-v0.2.0-s.tar.gz”

tencent-ailab-embed 100%[===================>]   6.68K  --.-KB/s  用时 0.04s     

2024-12-09 17:10:24 (162 KB/s) - 已保存 “tencent-ailab-embedding-zh-d100-v0.2.0-s.tar.gz” [6840/6840])

tar: Error opening archive: Unrecognized archive format


In [8]:
!mkdir output_data/

mkdir: output_data/: File exists


In [9]:
import os
import json
import math
import jieba
import dataclasses
import numpy as np
import logging
from dataclasses import field, dataclass
from gensim.models import KeyedVectors
from typing import List, Union, Dict, Any, Mapping, Optional

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

jieba.setLogLevel(logging.INFO)

In [10]:
@dataclass
class InputExample:

    guid: str
    text_a: str
    text_b: Optional[str] = None
    label: Optional[str] = None

    def to_json_string(self):
        return json.dumps(dataclasses.asdict(self), indent=2) + "\n"

In [11]:
@dataclass
class DataTrainingArguments:

    w2v_file: str = field(
        default='tencent-ailab-embedding-zh-d100-v0.2.0-s/tencent-ailab-embedding-zh-d100-v0.2.0-s.txt',
        metadata={'help': 'The pretrained word2vec model directory'}
    )
    data_dir: str = field(
        default='KUAKE-QQR',
        metadata={'help': 'The data directory'}
    )

    def __str__(self):
        self_as_dict = dataclasses.asdict(self)
        attrs_as_str = [f"{k}={v},\n" for k, v in sorted(self_as_dict.items())]
        return f"{self.__class__.__name__}(\n{''.join(attrs_as_str)})"
        
    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(dataclasses.asdict(self), indent=2) + "\n"

In [12]:
@dataclass
class TrainingArguments:

    output_dir: str = field(
        default='output_data/',
        metadata={'help': 'The output directory where the model predictions and checkpoints will be written.'}
    )
    train_batch_size: int = field(
        default=64,
        metadata={'help': 'batch size for training'}
    )
    eval_batch_size: int = field(
        default=64,
        metadata={'help': 'batch size for evaluation'}
    )
    num_train_epochs: int = field(
        default=27,
        metadata={"help": "The total number of training epochs"}
    )
    learning_rate: float = field(
        default=0.001,
        metadata={'help': '"The initial learning rate for AdamW.'}
    )
    weight_decay: float = field(
        default=5e-4,
        metadata={"help": "Weight decay for AdamW"}
    )
    logging_steps: int = field(
        default=50,
        metadata={'help': 'logging states every X updates steps.'}
    )
    eval_steps: int = field(
        default=100,
        metadata={'help': 'Run an evaluation every X steps.'}
    )
    device: str = field(
        default='cpu',
        metadata={"help": 'The device used for training'}
    )

    def __str__(self):
        self_as_dict = dataclasses.asdict(self)
        attrs_as_str = [f"{k}={v},\n" for k, v in sorted(self_as_dict.items())]
        return f"{self.__class__.__name__}(\n{''.join(attrs_as_str)})"
        
    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(dataclasses.asdict(self), indent=2) + "\n"

In [13]:
@dataclass
class ModelArguments:

    in_feat: int = field(
        default=100,
        metadata={'help': 'Size of input sample.'}
    )
    dropout_prob: float = field(
        default=0.1,
        metadata={'help': 'Dropout probability.'}
    )

    def __str__(self):
        self_as_dict = dataclasses.asdict(self)
        attrs_as_str = [f"{k}={v},\n" for k, v in sorted(self_as_dict.items())]
        return f"{self.__class__.__name__}(\n{''.join(attrs_as_str)})"
        
    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(dataclasses.asdict(self), indent=2) + "\n"

In [14]:
class QQRProcessor:
    TASK = 'KUAKE-QQR'

    def __init__(self, data_dir):
        self.task_dir = os.path.join(data_dir)

    def get_train_examples(self):
        return self._create_examples(os.path.join(self.task_dir, f'{self.TASK}_train.json'))

    def get_dev_examples(self):
        return self._create_examples(os.path.join(self.task_dir, f'{self.TASK}_dev.json'))

    def get_test_examples(self):
        return self._create_examples(os.path.join(self.task_dir, f'{self.TASK}_test.json'))

    def get_labels(self):
        return ["0", "1", "2"]

    def _create_examples(self, data_path):

        # 读入文件
        with open(data_path, 'r', encoding='utf-8') as f:
            samples = json.load(f)

        examples = []
        for sample in samples:
            guid = sample['id']
            text_a = sample['query1']
            text_b = sample['query2']
            label = sample.get('label', None)

            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))

        return examples

In [15]:
class QQRDataset(Dataset):

    def __init__(
        self,
        examples: List[InputExample],
        label_list: List[Union[str, int]],
        vocab_mapping: Dict,
        max_length: int = 64,
    ):
        super().__init__()

        self.examples = examples

        self.vocab_mapping = vocab_mapping
        self.max_length = max_length

        self.label2id = {label: idx for idx, label in enumerate(label_list)}
        self.id2label = {idx: label for idx, label in enumerate(label_list)}

    def __len__(self):
        return len(self.examples)

    def _tokenize(self, text):
        # 文本分词
        tokens = list(jieba.cut(text))

        token_ids = []
        for token in tokens:
            if token in self.vocab_mapping:
                # 如果当前词存在于词表，将词转换为词的ID.
                token_id = self.vocab_mapping[token]
                token_ids.append(token_id)
            else:
                # OOV情况处理
                # 如果该词为多字的词，将其拆分多个字，分别将这些字转换为相应的ID；
                # 如果该词为单字，则从词表中随机采样一个词，其ID作为该词的ID
                if len(token) > 1:
                    for t in list(token):
                        if t in self.vocab_mapping:
                            token_ids.append(self.vocab_mapping[t])
                        else:
                            token_ids.append(np.random.choice(len(self.vocab_mapping), 1)[0])
                else:
                    token_ids.append(np.random.choice(len(self.vocab_mapping), 1)[0])

        # 对文本进行填充或者截断
        token_ids, attention_mask = self._pad_truncate(token_ids)
        
        return token_ids, attention_mask

    def _pad_truncate(self, token_ids: List[int]):
        # attention_mask作为标识文本填充情况
        attention_mask = None

        # 如果文本长度（以词为单位）大于设定的阈值，则截断末尾部分；
        # 如果文本长度小于设定的阈值，则填充0
        if len(token_ids) > self.max_length:
            token_ids = token_ids[:self.max_length], 
            attention_mask = [1] * self.max_length
        else:
            attention_mask = [1] * len(token_ids)

            diff = self.max_length - len(token_ids)
            token_ids.extend([0] * diff)
            attention_mask.extend([0] * diff)

        return token_ids, attention_mask        
        
    def __getitem__(self, index):
        
        example = self.examples[index]
        label = None
        # if example.label is not None:
        #     label = self.label2id[example.label]
        if example.label is None or example.label == '':
            label = 0  # 测试集默认标签
        else:
            if example.label == 'NA':
                example.label = '0'
            label = self.label2id[example.label]

        # tokenize
        text_a_token_ids, text_a_attention_mask = self._tokenize(example.text_a)
        text_b_token_ids, text_b_attention_mask = self._tokenize(example.text_b)

        return {'text_a_input_ids': text_a_token_ids, 'text_b_input_ids': text_b_token_ids, 
                'text_a_attention_mask': text_a_attention_mask, 'text_b_attention_mask': text_b_attention_mask, 'label': label}


class DataCollator:

    def __call__(self, features: List[Dict[str, Any]]):
        
        # 将一个batch内的样本输入转换为Tensor
        
        text_a_input_ids = []
        text_b_input_ids = []
        text_a_attention_mask = []
        text_b_attention_mask = []
        labels = []
        for item in features:
            text_a_input_ids.append(item['text_a_input_ids'])
            text_b_input_ids.append(item['text_b_input_ids'])
            text_a_attention_mask.append(item['text_a_attention_mask'])
            text_b_attention_mask.append(item['text_b_attention_mask'])
            if item['label'] is not None:
                labels.append(item['label'])
        
        text_a_input_ids = torch.tensor(text_a_input_ids, dtype=torch.long)
        text_b_input_ids = torch.tensor(text_b_input_ids, dtype=torch.long)
        text_a_attention_mask = torch.tensor(text_a_attention_mask, dtype=torch.bool)
        text_b_attention_mask = torch.tensor(text_b_attention_mask, dtype=torch.bool)
        if len(labels) > 0:
            labels = torch.tensor(labels, dtype=torch.long)
        else:
            labels = None

        return {'text_a_input_ids': text_a_input_ids, 'text_b_input_ids': text_b_input_ids, 
                'text_a_attention_mask': text_a_attention_mask, 'text_b_attention_mask': text_b_attention_mask, 'labels': labels}

In [16]:
class Encoder(nn.Module):

    def __init__(self, in_feat: int = 100, dropout_prob: float = 0.1):
        super().__init__()

        self.dense1 = nn.Linear(in_feat, in_feat)
        self.dense2 = nn.Linear(in_feat, in_feat)
        self.dropout = nn.Dropout(dropout_prob)
        self.act = nn.Tanh()

    def forward(self, token_embeds, attention_mask):
        batch_size = token_embeds.size(0)
        
        # 进行Mean pooling操作，即简单地将文本的词向量求和
        x = torch.stack([token_embeds[i, attention_mask[i, :], :].sum(dim=0) for i in range(batch_size)], dim=0)

        x = self.act(self.dense1(x))
        x = self.act(self.dense2(self.dropout(x)))

        return x


class Classifier(nn.Module):

    def __init__(self, in_feat, num_labels: int, dropout_prob: float = 0.1):
        super().__init__()

        self.dense1 = nn.Linear(in_feat, in_feat // 2)
        self.dense2 = nn.Linear(in_feat // 2, num_labels)
        self.act = nn.Tanh()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):

        x = self.act(self.dense1(self.dropout(x)))
        x = self.dense2(self.dropout(x))

        return x


class SemNN(nn.Module):

    def __init__(
        self,
        in_feat: int = 100,
        num_labels: int = 3,
        dropout_prob: float = 0.1,
        w2v_state_dict: torch.Tensor = None,
        vocab_size: int = None,
        word_embedding_dim: int = None
    ):
        super().__init__()

        self.num_labels = num_labels
        self._init_word_embedding(w2v_state_dict, vocab_size, word_embedding_dim)
        
        self.encoder = Encoder(in_feat=in_feat)
        self.classifier = Classifier(in_feat=2*in_feat, num_labels=num_labels, dropout_prob=dropout_prob)

    def _init_word_embedding(self, state_dict=None, vocab_size=None, word_embedding_dim=None):
        if state_dict is None:
            self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim, padding_idx=0)
        else:
            # 默认载入预训练好的词向量（且固定词向量），并将其第一个词作为填充词（以及其对应向量设为零向量）
            state_dict = torch.tensor(state_dict.vectors, dtype=torch.float32)
            state_dict[0] = torch.zeros(state_dict.size(-1))
            self.word_embedding = nn.Embedding.from_pretrained(state_dict, freeze=True, padding_idx=0)

    def forward(
        self, 
        text_a_input_ids,
        text_b_input_ids,
        text_a_attention_mask,
        text_b_attention_mask, 
        labels=None
    ):
        # 将两个query的词ID转换为其相应的词向量
        text_a_vecs = self.word_embedding(text_a_input_ids)
        text_b_vecs = self.word_embedding(text_b_input_ids)

        # 通过Encoder得到两个query的向量表示
        text_a_vec = self.encoder(text_a_vecs, text_a_attention_mask)
        text_b_vec = self.encoder(text_b_vecs, text_b_attention_mask)

        # 拼接两个Query的表示，再输入到分类器中
        pooler_output = torch.cat([text_a_vec, text_b_vec], dim=-1)
        logits = self.classifier(pooler_output)
        
        # 训练过程中的Loss计算
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits

In [17]:
def create_optimizer_and_lr_scheduler(
    args: TrainingArguments,
    model: nn.Module
):
    # 构建优化器
    optimizer = AdamW(
        model.parameters(), 
        lr=args.learning_rate,
        weight_decay=args.weight_decay,
    )
    # 构建学习率调度器
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=3, T_mult=2, eta_min=1e-5)

    return optimizer, lr_scheduler


def _prepare_input(data: Union[torch.Tensor, Any], device: str = 'cuda'):
    # 将准备输入模型中的数据转到GPU上
    if isinstance(data, Mapping):
        return type(data)({k: _prepare_input(v, device) for k, v in data.items()})
    elif isinstance(data, (tuple, list)):
        return type(data)(_prepare_input(v, device) for v in data)
    elif isinstance(data, torch.Tensor):
        kwargs = dict(device=device)
        return data.to(**kwargs)
    return data


def simple_accuracy(preds, labels):

    return (preds == labels).mean()
    

def evaluate(
    args: TrainingArguments,
    model: nn.Module,
    eval_dataloader
):
    model.eval()
    loss_list = []
    preds_list = []
    labels_list = []

    for item in eval_dataloader:
        inputs = _prepare_input(item, device=args.device)

        with torch.no_grad():
            outputs = model(**inputs)
            loss = outputs[0]
            loss_list.append(loss.detach().cpu().item())

            preds = torch.argmax(outputs[1].cpu(), dim=-1).numpy()
            preds_list.append(preds)

            labels_list.append(inputs['labels'].cpu().numpy())
    
    preds = np.concatenate(preds_list, axis=0)
    labels = np.concatenate(labels_list, axis=0)
    loss = np.mean(loss_list)
    accuracy = simple_accuracy(preds, labels)

    model.train()

    return loss, accuracy


def train(
    args: TrainingArguments,
    model: nn.Module,
    train_dataset,
    dev_dataset,
    data_collator,
):

    # initialize dataloader
    train_dataloader = DataLoader(
        dataset=train_dataset, 
        batch_size=args.train_batch_size,
        shuffle=True,
        collate_fn=data_collator
    )
    dev_dataloader = DataLoader(
        dataset=dev_dataset,
        batch_size=args.eval_batch_size,
        shuffle=False,
        collate_fn=data_collator
    )

    num_examples = len(train_dataloader.dataset)
    num_update_steps_per_epoch = len(train_dataloader)
    num_update_steps_per_epoch = max(num_update_steps_per_epoch, 1)
    
    max_steps = math.ceil(args.num_train_epochs * num_update_steps_per_epoch)
    num_train_epochs = math.ceil(args.num_train_epochs)
    num_train_samples = len(train_dataset) * args.num_train_epochs

    optimizer, lr_scheduler = create_optimizer_and_lr_scheduler(args, model)

    print("***** Running training *****")
    print(f"  Num examples = {num_examples}")
    print(f"  Num Epochs = {args.num_train_epochs}")
    print(f"  Instantaneous batch size per device = {args.train_batch_size}")
    print(f"  Total train batch size (w. parallel, distributed & accumulation) = {args.train_batch_size}")
    print(f"  Total optimization steps = {max_steps}")

    model.zero_grad()
    model.train()
    global_steps = 0

    best_metric = 0.0
    best_steps = -1

    for epoch in range(num_train_epochs):
        for step, item in enumerate(train_dataloader):
            inputs = _prepare_input(item, device=args.device)
            outputs = model(**inputs)
            loss = outputs[0]

            loss.backward()
            optimizer.step()
            lr_scheduler.step(epoch + step / num_update_steps_per_epoch)

            model.zero_grad()
            global_steps += 1

            if global_steps % args.logging_steps == 0:
                print(f'Training: Epoch {epoch + 1}/{num_train_epochs} - Step {(step + 1)} - Loss {loss}')

            if global_steps % args.eval_steps == 0:
                
                loss, acc = evaluate(args, model, dev_dataloader)
                print(f'Evaluation: Epoch {epoch + 1}/{num_train_epochs} - Step {(global_steps + 1)} - Loss {loss} - Accuracy {acc}')

                if acc > best_metric:
                    best_metric = acc
                    best_steps = global_steps
                    
                    saved_path = os.path.join(args.output_dir, f'checkpoint-{best_steps}.pt')
                    torch.save(model.state_dict(), saved_path)

    return best_steps, best_metric

In [ ]:
def predict(
    args: TrainingArguments,
    model: nn.Module,
    test_dataset,
    data_collator
):
    # 初始化DataLoader
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=args.eval_batch_size, 
        shuffle=False,
        collate_fn=data_collator
    )

    print("***** Running prediction *****")
    print(f"  Num examples = {len(test_dataset)}")
    print(f"  Batch size = {args.eval_batch_size}")
    
    model.eval()
    predictions = []

    # 逐批次预测
    for batch in test_dataloader:
        batch = _prepare_input(batch, device=args.device)
        
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs[1] if isinstance(outputs, tuple) else outputs
            pred = torch.argmax(logits, dim=-1)
            predictions.extend(pred.cpu().numpy().tolist())
            
    model.train()
    
    # 将数字标签映射回原始标签
    predictions = [test_dataset.id2label[pred] for pred in predictions]
    
    print(f"Prediction finished, total predictions: {len(predictions)}")
    return predictions

def generate_commit(output_dir, task_name, test_dataset, predictions):
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 整合预测结果
    results = []
    for example, pred_label in zip(test_dataset.examples, predictions):
        result = {
            "id": example.guid,
            "query1": example.text_a,
            "query2": example.text_b,
            "label": pred_label
        }
        results.append(result)
    
    # 保存结果
    output_file = os.path.join(output_dir, f'{task_name}_test.json') 
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"Results saved to {output_file}")

In [19]:
import time

data_args = DataTrainingArguments()
training_args = TrainingArguments()
model_args = ModelArguments()
print(data_args)
print(training_args)
print(model_args)

#载入词向量
w2v_model = KeyedVectors.load_word2vec_format(data_args.w2v_file, binary=False)

DataTrainingArguments(
data_dir=KUAKE-QQR,
w2v_file=tencent-ailab-embedding-zh-d100-v0.2.0-s/tencent-ailab-embedding-zh-d100-v0.2.0-s.txt,
)
TrainingArguments(
device=cpu,
eval_batch_size=64,
eval_steps=100,
learning_rate=0.001,
logging_steps=50,
num_train_epochs=27,
output_dir=output_data/,
train_batch_size=64,
weight_decay=0.0005,
)
ModelArguments(
dropout_prob=0.1,
in_feat=100,
)


In [20]:
processor = QQRProcessor(data_dir=data_args.data_dir)

train_dataset = QQRDataset(
    processor.get_train_examples(), 
    processor.get_labels(),
    vocab_mapping=w2v_model.key_to_index,
    max_length=32
)
eval_dataset = QQRDataset(
    processor.get_dev_examples(),
    processor.get_labels(),
    vocab_mapping=w2v_model.key_to_index,
    max_length=32
)
test_dataset = QQRDataset(
    processor.get_test_examples(),
    processor.get_labels(),
    vocab_mapping=w2v_model.key_to_index,
    max_length=32
)

data_collator = DataCollator()

In [21]:
# 创建输出结果（模型、参数、预测结果）的文件夹
model_name = f'semnn-{str(int(time.time()))}'
training_args.output_dir = os.path.join(training_args.output_dir, model_name)
if not os.path.exists(training_args.output_dir):
    os.makedirs(training_args.output_dir, exist_ok=True)

In [22]:
# 初始化模型
print('Initialize model')
model = SemNN(
    in_feat=model_args.in_feat, 
    num_labels=len(processor.get_labels()), 
    dropout_prob=model_args.dropout_prob,
    w2v_state_dict=w2v_model,
)
model.to(training_args.device)

Initialize model


SemNN(
  (word_embedding): Embedding(2000000, 100, padding_idx=0)
  (encoder): Encoder(
    (dense1): Linear(in_features=100, out_features=100, bias=True)
    (dense2): Linear(in_features=100, out_features=100, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (act): Tanh()
  )
  (classifier): Classifier(
    (dense1): Linear(in_features=200, out_features=100, bias=True)
    (dense2): Linear(in_features=100, out_features=3, bias=True)
    (act): Tanh()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [23]:
# 训练模型
print('Training...')
best_steps, best_metric = train(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    dev_dataset=eval_dataset,
    data_collator=data_collator
)

print(f'Training Finished! Best step - {best_steps} - Best accuracy {best_metric}')

best_model_path = os.path.join(training_args.output_dir, f'checkpoint-{best_steps}.pt')
model = SemNN(
    in_feat=model_args.in_feat, 
    num_labels=len(processor.get_labels()), 
    dropout_prob=model_args.dropout_prob,
    w2v_state_dict=w2v_model,
)
model.load_state_dict(torch.load(best_model_path, map_location='cpu'))
model.to(training_args.device)

# 保存最佳模型及超参数
torch.save(model.state_dict(), os.path.join(training_args.output_dir, 'pytorch_model.bin'))
torch.save(training_args, os.path.join(training_args.output_dir, 'training_args.bin'))

# 预测及生成预测结果（供提交到平台）
preds = predict(training_args, model, test_dataset, data_collator)
generate_commit(training_args.output_dir, processor.TASK, test_dataset, preds)

Training...
***** Running training *****
  Num examples = 15000
  Num Epochs = 27
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Total optimization steps = 6345
Training: Epoch 1/27 - Step 50 - Loss 0.8645390868186951
Training: Epoch 1/27 - Step 100 - Loss 0.8376542925834656
Evaluation: Epoch 1/27 - Step 101 - Loss 0.8075150513648987 - Accuracy 0.664375
Training: Epoch 1/27 - Step 150 - Loss 0.8339117169380188
Training: Epoch 1/27 - Step 200 - Loss 0.7950250506401062
Evaluation: Epoch 1/27 - Step 201 - Loss 0.8182623481750488 - Accuracy 0.639375
Training: Epoch 2/27 - Step 15 - Loss 0.6882098317146301
Training: Epoch 2/27 - Step 65 - Loss 0.9125463962554932
Evaluation: Epoch 2/27 - Step 301 - Loss 0.8144591212272644 - Accuracy 0.63875
Training: Epoch 2/27 - Step 115 - Loss 0.6464906930923462
Training: Epoch 2/27 - Step 165 - Loss 0.7004722356796265
Evaluation: Epoch 2/27 - Step 401 - Loss 0.8318505442142486 - Accurac

KeyboardInterrupt: 